In [148]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import re
import json

In [188]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

# Set the number of scrolls and the pause time between scrolls
NUMBER_OF_SCROLLS = 5
SCROLL_PAUSE_TIME = 3  # in seconds

# URL of the page to scrape
url = 'https://www.qantas.com/hotels/properties/41988?adults=2&checkIn=2024-01-31&checkOut=2024-02-01&children=0&infants=0&location=Vienna%2C%20Austria&page=1&payWith=cash&searchType=list&sortBy=popularity'

# Initialize the WebDriver (this example uses Firefox)
# Make sure geckodriver is in your PATH or specify the path directly
driver = webdriver.Firefox()

# Open the URL
driver.get(url)

# Scroll down the page
for i in range(NUMBER_OF_SCROLLS):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)

# Get the page source and close the browser
html_content = driver.page_source
driver.quit()

In [182]:
# Assuming html_content is your variable containing the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all room name elements
room_name_elements = soup.find_all("h3", class_ =lambda x: x and "Heading-Heading-Text" in x)
# Iterate over each room name element and find the corresponding price
for room_name_element in room_name_elements:
    room_name = room_name_element.get_text(strip=True)
    if not room_name_element.find_next(class_= re.compile('BackgroundImage')):
        break


    # Assuming the price is within a close proximity in the HTML structure
    price_element = room_name_element.find_next('span', {'data-testid': 'amount'})
    if price_element:
        price = price_element.get_text(strip=True)
        #print(list(price_element.parents)[15].prettify())


    print(f'Room: {room_name}, Price: {price}')






Room: Room prices from, Price: 759
Room: Room prices from, Price: 759
Room: Premium Room, 1 King Bed, Park View, Price: 759
Room: Premium Room, 1 King Bed, Park View, Price: 759
Room: Internet Included - Non-refundable, Price: 759
Room: Internet Included, Price: 884
Room: Internet Included, Price: 893
Room: Internet Included, Breakfast Included, Price: 967
Room: Premium Room, 1 King Bed, Corner (Panorama), Price: 773
Room: Premium Room, 1 King Bed, Corner (Panorama), Price: 773
Room: Internet Included - Non-refundable, Price: 773
Room: Internet Included, Price: 900
Room: Internet Included, Price: 910
Room: Internet Included, Breakfast Included, Price: 982
Room: Executive Room, 2 Twin Beds, Price: 888
Room: Executive Room, 2 Twin Beds, Price: 888
Room: Internet Included - Non-refundable, Price: 888
Room: Internet Included, Price: 965
Room: Internet Included, Price: 975
Room: Internet Included, Breakfast Included, Price: 1,047
Room: Junior Suite, 1 King Bed (Panorama), Price: 996
Room: J

In [183]:
soup = BeautifulSoup(html_content, 'html.parser')
buttons = soup.find('button', string='View room details')

hotel_accomodations = []

boxes_container = list(buttons.parents)[5] # Getting html boxes of rooms by going 5 parents up the tree
for box in boxes_container:
    room_name = box.find('h3')
    if room_name:
        room_name = room_name.get_text(strip=True)
    
        # Find the price within the same container
        price = box.find_next('span', {'data-testid': 'amount'})
        if price:
            price = price.get_text(strip=True)
        
        rooms  = []
        guests = None
        for i in box.find_all('h3', class_= lambda x: x and "Heading-Heading-Text" in x):
            minibox = i.parent.parent.parent
            price = minibox.find('span',  {'data-testid': 'amount'})
            

            
            topdeal = minibox.find('span', string = 'Top Deal') is not None
            if price is None:
                continue
            guests = guests or minibox.find('span', string = re.compile('guests'))
            currency_element = next(price.parent.parent.parent.find_next_sibling().children)
            rooms.append({
                'rate_name': i.text,
                "price": price.text,
                'currency': currency_element.text,
                'topdeal':topdeal,
                'guests': guests.text.split(" ")[0],
                
                
                
            })
        hotel_accomodations.append({'room_name': room_name,
                                    'data': rooms
                                   })
print(json.dumps(hotel_accomodations, indent = 2))


[
  {
    "room_name": "Premium Room, 1 King Bed, Park View",
    "data": [
      {
        "rate_name": "Premium Room, 1 King Bed, Park View",
        "price": "759",
        "currency": " AUD",
        "topdeal": true,
        "guests": "2"
      },
      {
        "rate_name": "Internet Included",
        "price": "884",
        "currency": " AUD",
        "topdeal": false,
        "guests": "2"
      },
      {
        "rate_name": "Internet Included",
        "price": "893",
        "currency": " AUD",
        "topdeal": false,
        "guests": "2"
      },
      {
        "rate_name": "Internet Included, Breakfast Included",
        "price": "967",
        "currency": " AUD",
        "topdeal": false,
        "guests": "2"
      }
    ]
  },
  {
    "room_name": "Premium Room, 1 King Bed, Corner (Panorama)",
    "data": [
      {
        "rate_name": "Premium Room, 1 King Bed, Corner (Panorama)",
        "price": "773",
        "currency": " AUD",
        "topdeal": true,
      

In [190]:
from lxml import etree
import re
import json

# Parse the HTML content
tree = etree.HTML(html_content)

hotel_accommodations = []
processed_rooms = set()  # Set to keep track of processed room names

# Find buttons with specific text
buttons = tree.xpath("//button[contains(text(), 'View room details')]")

# Navigate to the container that holds the room details
for button in buttons:
    box_container = button.getparent()
    for _ in range(5):  # Adjust the range as needed to reach the correct parent
        box_container = box_container.getparent()

    for box in box_container:
        room_elements = box.xpath(".//h3")
        if room_elements:
            room_name = room_elements[0].text.strip()

            # Skip if this room has already been processed
            if room_name in processed_rooms:
                continue
            processed_rooms.add(room_name)

            price_elements = box.xpath(".//span[@data-testid='amount']")
            #price = price_elements[0].text.strip() if price_elements else None

            rooms = []
            guests = None

            for heading in box.xpath(".//h3[contains(@class, 'Heading-Heading-Text')]"):
                mini_box = heading.getparent()
                for _ in range(2):  # Adjust as necessary
                    mini_box = mini_box.getparent()

                price_elements = mini_box.xpath(".//span[@data-testid='amount']")
                top_deal_elements = mini_box.xpath(".//span[contains(text(), 'Top Deal')]")

                if price_elements == []:
                    continue
                    
                price_elements = price_elements[0]

                guests_elements = mini_box.xpath(".//span[contains(text(), 'guests')]")
                guests = guests or (guests_elements[0].text.split(" ")[0] if guests_elements else None)

                currency_elements = price_elements.xpath("../../../following-sibling::*[1]/*")
                currency = currency_elements[0].text.strip() if currency_elements else "Unknown"
                
                rooms.append({
                    'rate_name': heading.text.strip(),
                    'price': "".join (price_elements.itertext()).strip(),
                    'currency': currency,
                    'topdeal': bool(top_deal_elements),
                    'guests': guests
                })

            hotel_accommodations.append({'room_name': room_name, 'data': rooms})

print(json.dumps(hotel_accommodations, indent=2))



[
  {
    "room_name": "Premium Room, 1 King Bed, Park View",
    "data": [
      {
        "rate_name": "Premium Room, 1 King Bed, Park View",
        "price": "759",
        "currency": "AUD",
        "topdeal": true,
        "guests": "2"
      },
      {
        "rate_name": "Internet Included",
        "price": "884",
        "currency": "AUD",
        "topdeal": false,
        "guests": "2"
      },
      {
        "rate_name": "Internet Included",
        "price": "893",
        "currency": "AUD",
        "topdeal": false,
        "guests": "2"
      },
      {
        "rate_name": "Internet Included, Breakfast Included",
        "price": "967",
        "currency": "AUD",
        "topdeal": false,
        "guests": "2"
      }
    ]
  },
  {
    "room_name": "Premium Room, 1 King Bed, Corner (Panorama)",
    "data": [
      {
        "rate_name": "Premium Room, 1 King Bed, Corner (Panorama)",
        "price": "773",
        "currency": "AUD",
        "topdeal": true,
        "gu